In [2]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dogs are great companions",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Cats are independent pets",
        metadata={"source": "mammal-pets-doc"}
    ),
    Document(
        page_content="Goldfish are popular pets for beginners",
        metadata={"source": "fish-pets-doc"}
    )
]


In [4]:
import os 
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")
llm=ChatGroq(groq_api_key=groq_api_key,model="Llama3-8b-8192")

In [6]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [7]:
# VectorStores
from langchain_chroma import Chroma
vectorstore= Chroma.from_documents(documents,embedding=embeddings)

In [8]:
vectorstore.similarity_search("cat")

[Document(id='b805e60a-50e9-444a-a450-3ba76fb1622c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets'),
 Document(id='e9c31476-c3ee-45f1-9143-73911d1f08f7', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions'),
 Document(id='c29ef46f-c734-4cc0-8ef2-f39fce054f37', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners')]

In [9]:
# Async query
await vectorstore.asimilarity_search("cat")

[Document(id='b805e60a-50e9-444a-a450-3ba76fb1622c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets'),
 Document(id='e9c31476-c3ee-45f1-9143-73911d1f08f7', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions'),
 Document(id='c29ef46f-c734-4cc0-8ef2-f39fce054f37', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners')]

In [10]:
vectorstore.similarity_search_with_score("cat")

[(Document(id='b805e60a-50e9-444a-a450-3ba76fb1622c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets'),
  0.8409360647201538),
 (Document(id='e9c31476-c3ee-45f1-9143-73911d1f08f7', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions'),
  1.3763905763626099),
 (Document(id='c29ef46f-c734-4cc0-8ef2-f39fce054f37', metadata={'source': 'fish-pets-doc'}, page_content='Goldfish are popular pets for beginners'),
  1.6656098365783691)]

In [13]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda
retriever=RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(["cat","dog"])

[[Document(id='b805e60a-50e9-444a-a450-3ba76fb1622c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets')],
 [Document(id='e9c31476-c3ee-45f1-9143-73911d1f08f7', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions')]]

In [14]:
retriever=vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriever.batch(["cat","dog"])

[[Document(id='b805e60a-50e9-444a-a450-3ba76fb1622c', metadata={'source': 'mammal-pets-doc'}, page_content='Cats are independent pets')],
 [Document(id='e9c31476-c3ee-45f1-9143-73911d1f08f7', metadata={'source': 'mammal-pets-doc'}, page_content='Dogs are great companions')]]

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
message="""
Answer this question using the provided context only.
{question}
Context:
{context}
"""
prompt=ChatPromptTemplate.from_messages([("human",message)])
rag_chain={"context":retriever,"question":RunnablePassthrough()}|prompt|llm
response=rag_chain.invoke("tell me about dogs")
print(response.content)

Dogs are great companions.
